# EXERCISE 6
Number 2 – CNN with Muffin vs Chihuahua Dataset


In [4]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers
import numpy as np

In [7]:
# DATASET DIRECTORY CONFIGURATION
train_dir = "train"  
test_dir = "test"    

In [8]:
# IMAGE PARAMETERS
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [9]:
# DATA PREPROCESSING & AUGMENTATION
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [10]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

c:\Users\melis\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
# Configure the model optimizers, loss function, and metrics
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 85s 704ms/step - accuracy: 0.6940 - loss: 0.5952 - val_accuracy: 0.7365 - val_loss: 0.5268
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 340ms/step - accuracy: 0.8068 - loss: 0.4486 - val_accuracy: 0.7735 - val_loss: 0.4984
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 64s 541ms/step - accuracy: 0.8345 - loss: 0.3815 - val_accuracy: 0.8603 - val_loss: 0.3722
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 62s 526ms/step - accuracy: 0.8516 - loss: 0.3536 - val_accuracy: 0.8857 - val_loss: 0.2915
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 345ms/step - accuracy: 0.8442 - loss: 0.3639 - val_accuracy: 0.8910 - val_loss: 0.3062
Epoch 6/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 343ms/step - accuracy: 0.8532 - loss: 0.3478 - val_accuracy: 0.8815 - val_loss: 0.2753
Epoch 7/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 346ms/step - accuracy: 0.8685 - loss: 0.3304 - val_accuracy: 0.9111 - val_loss: 0.2383
Epoch 8/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 41s 344ms/step - accuracy: 0.8783 - loss: 0

In [12]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 174ms/step - accuracy: 0.4628 - loss: 0.6958
Test Accuracy: 0.4628378450870514


In [9]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [16]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    if pred >= 0.5:
        label = "Muffin"
        confidence = pred
    else:
        label = "Chihuahua"
        confidence = 1.0 - pred
    print(f"Prediction: {label} (confidence: {confidence:.2f})")

In [19]:
# Example usage:
predict_image("test/chihuahua/img_0_18.jpg")
predict_image("test/muffin/img_0_10.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step


Prediction: Chihuahua (confidence: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Prediction: Muffin (confidence: 1.00)
